In [1]:
import bql
import bqplot as bqp
import pandas as pd

bq = bql.Service()

In [2]:
Members = bq.univ.members('LUACTRUU Index')

dates = bq.func.range('2019-12-31','2020-04-10')

spread = bq.func.dropna(bq.data.spread(dates=dates))
current_spread = bq.func.last(spread)
long_comp_name = bq.data.long_comp_name()

z_score = bq.func.z_score(spread)
current_z_score = bq.func.last(z_score)

fields = {'Name': long_comp_name, 'Current Spread': current_spread, 'Current Z Score': current_z_score}

rank = bq.func.grouprank(current_z_score)

Filtered_Members = bq.univ.filter(Members,rank<=200)

request = bql.Request(Filtered_Members,fields)
response = bq.execute(request)

**Examine `response` object**

In [3]:
#the full df for each field
for res in response:
    print(res.df().head())

                                                            Name
ID                                                              
AN267754 Corp  National Rural Utilities Cooperative Finance Corp
EH502090 Corp                      Public Service Co of Colorado
EI080338 Corp                             Boston Scientific Corp
EJ409739 Corp                                CommonSpirit Health
EK276667 Corp                     Corporate Office Properties LP
                    DATE  Current Spread
ID                                      
AN267754 Corp 2020-04-09      235.445679
EH502090 Corp 2020-04-09      295.915192
EI080338 Corp 2020-04-09      282.963989
EJ409739 Corp 2020-04-09      422.861877
EK276667 Corp 2020-04-09      253.987640
                    DATE  Current Z Score
ID                                       
AN267754 Corp 2020-04-09         2.197980
EH502090 Corp 2020-04-09         2.326671
EI080338 Corp 2020-04-09         2.130137
EJ409739 Corp 2020-04-09         2.670720
EK2766

### DF Merge Ninjitsu method 1

In [3]:
#combine the multi-dataframe response into one df
tbl = pd.DataFrame({r.name:r.df()[r.name] for r in response})
tbl.head()

,Name,Current Spread,Current Z Score
ID,,,
AN267754 Corp,National Rural Utilities Cooperative Finance Corp,235.445679,2.197980
EH502090 Corp,Public Service Co of Colorado,295.915192,2.326671
EI080338 Corp,Boston Scientific Corp,282.963989,2.130137
EJ409739 Corp,CommonSpirit Health,422.861877,2.670720
EK276667 Corp,Corporate Office Properties LP,253.987640,2.510107


### DF Merge Ninjitsu method 2

In [4]:
#Function to convert the result of a BQL query where meta data for each field isn't uniform
from functools import reduce
import pandas as pd

def merge_dfs(response):
    df_l = [x.df().reset_index() for x in list(response)]
    newdf = reduce(lambda left,right: pd.merge(left,right,on=['ID']), df_l)
    return newdf

data = merge_dfs(response)

In [5]:
#dataframe = bql.combined_df(response)
data.sample(20)

,ID,Name,DATE_x,Current Spread,DATE_y,Current Z Score
68,UV388933 Corp,Spirit Airlines Pass Through Trust 2015-1A,2020-04-09,721.310059,2020-04-09,2.947659
13,DD101122 Corp,Weyerhaeuser Co,2020-04-09,336.082001,2020-04-09,2.234621
38,EJ618909 Corp,Essex Portfolio LP,2020-04-09,354.798737,2020-04-09,3.584018
61,AM969171 Corp,Ventas Realty LP,2020-04-09,290.914551,2020-04-09,2.649100
149,AO296348 Corp,ORIX Corp,2020-04-09,318.270996,2020-04-09,2.514680
28,AQ214314 Corp,Healthcare Realty Trust Inc,2020-04-09,344.050476,2020-04-09,2.433353
128,QJ131551 Corp,Kimco Realty Corp,2020-04-09,274.124176,2020-04-09,2.548359
195,ED939561 Corp,El Paso Electric Co,2020-04-09,300.447235,2020-04-09,4.145675
62,EK796000 Corp,BorgWarner Inc,2020-04-09,406.252441,2020-04-09,2.063051
170,EK600613 Corp,Duke Realty LP,2020-04-09,352.726868,2020-04-09,2.881579


In [6]:
data.drop(['DATE_y'], axis=1, inplace=True)
data.rename(columns = {'DATE_x': 'DATE'}, inplace=True)
data.head()

,ID,Name,DATE,Current Spread,Current Z Score
0,AN267754 Corp,National Rural Utilities Cooperative Finance Corp,2020-04-09,235.445679,2.197980
1,EH502090 Corp,Public Service Co of Colorado,2020-04-09,295.915192,2.326671
2,EI080338 Corp,Boston Scientific Corp,2020-04-09,282.963989,2.130137
3,EJ409739 Corp,CommonSpirit Health,2020-04-09,422.861877,2.670720
4,EK276667 Corp,Corporate Office Properties LP,2020-04-09,253.987640,2.510107


In [7]:
#dataframe.reset_index(inplace=True)

#dataframe.sample(n=20)
dataframe = data


### <span style='color:orange'> Snippet </span> _(h/t Sal Failla)_

In [12]:
dataframe.sample(10)

,ID,Name,DATE,Current Spread,Current Z Score
182,AZ327079 Corp,UDR Inc,2020-04-09,292.856598,2.114133
121,AM201825 Corp,Regency Centers LP,2020-04-09,378.631470,2.125270
83,AX754628 Corp,Ingersoll-Rand Luxembourg Finance SA,2020-04-09,255.582870,2.170313
65,AZ132511 Corp,American Campus Communities Operating Partners...,2020-04-09,332.856689,2.429392
53,LW430737 Corp,Evergy Kansas Central Inc,2020-04-09,149.812500,2.419672
98,BG022085 Corp,Oshkosh Corp,2020-04-09,347.440735,2.209044
151,AM276086 Corp,Children's Hospital Corp/The,2020-04-09,215.947159,2.179554
123,AL636052 Corp,EPR Properties,2020-04-09,669.801147,2.292993
166,EK744869 Corp,MPLX LP,2020-04-09,919.466980,2.270598
54,AX316775 Corp,Ventas Realty LP,2020-04-09,430.173279,2.194352


In [11]:
import bqplot as bqp
from bqplot.interacts import BrushSelector
import pandas as pd
import numpy as np
from ipywidgets import Dropdown, HBox, VBox, HTML
import bqwidgets as bqw


# Create scales
scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()

# Create marks
mark_scatter = bqp.Scatter(x=dataframe[dataframe.columns[3]],
                           y=dataframe[dataframe.columns[4]],
                           scales={'x': scale_x, 'y': scale_y},
                           default_size=48,
                           colors=['#1B84ED'])

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label=dataframe.columns[3])
axis_y = bqp.Axis(scale=scale_y,
                  orientation='vertical',
                  tick_format='0.0f',
                  label=dataframe.columns[4])

# Create selector
selector = BrushSelector(x_scale=scale_x,
                         y_scale=scale_y,
                         marks=[mark_scatter])

# Create Figure
figure = bqp.Figure(marks=[mark_scatter],
                    axes=[axis_x, axis_y],
                    animation_duration=500,
                    layout={'width':'99%', 'height':'400px'},
                    padding_x=0.05,
                    title='Interactive scatter plot with linked datagrid',
                    title_style={'font-size': '22px'},
                    padding_y=0.05,
                    interaction=selector,
                    fig_margin={'top': 50, 'bottom': 60,
                                'left': 50, 'right':30})

# Create dropown widgets
dropdown_x = Dropdown(description='X axis',
                      options=dataframe.columns,
                      value=dataframe.columns[3])
dropdown_y = Dropdown(description='Y axis',
                      options=dataframe.columns,
                      value=dataframe.columns[4])

# Define callback function for dropdown widgets
def update_plot(evt):
    if evt is not None:
        new_value = evt['new']
        if evt['owner'] == dropdown_x:
            mark_scatter.x = dataframe[new_value]
            axis_x.label = new_value
        elif evt['owner'] == dropdown_y:
            mark_scatter.y = dataframe[new_value]
            axis_y.label = new_value


# Define callback function for selections
def on_select(evt):
    if evt is not None and evt['new'] is not None:
        indices = evt['new']
        datagrid.data = dataframe.iloc[indices].reset_index()

# Bind callback to the dropdown widgets
dropdown_x.observe(update_plot, names=['value'])
dropdown_y.observe(update_plot, names=['value'])
mark_scatter.observe(on_select, names=['selected'])

# Create datagrid
datagrid = bqw.DataGrid(data=dataframe, layout={'height': '300px'})

# Create Box containers
widget_box = HBox([dropdown_x, dropdown_y], layout={'margin': '10px'})
app_container = VBox([figure, widget_box, datagrid],
                     layout={'width':'100%'})

# Display the visualization
app_container
